In [319]:
# importacion de librerias

import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as sft
import matplotlib.pyplot as plt

from PIL import Image as im
import skimage.filters.rank as se
import skimage as sk


In [302]:
#Aqui se encuentran los programas para reconstruir

def convft(U,H,dx):
    """
    Realiza la convolucion 2D de las matrices U,H usando la transformada rapida de Fourier. U, H deben tener la misma forma.
    
    Parameters
    ----------
    U : Array Numpy
        Uno de las matrices a convolucionar
    H : Array Numpy
        Otra de las matrices a convolucionar
    dx : Float
        Tamaño de muestreo de las funciones a convolucionar. Necesario para preservar escala.

    Returns
    -------
    U2
        Arreglo de numpy con la convolucio de U y H

    """
    U2=sft.fft2(sft.fftshift(U))*(dx**2)
    U2=H*U2
    U2=sft.ifftshift(sft.ifft2(U2)*(1/dx**2))
    return U2

def circ(x): # FUNCION circulo EN 2D
        s=np.zeros(x.shape)
        s[(x)<1]=1.0
        return s
    

def tajada(M1,M2,x,y):
    """
    Esta funcion permite recortar una seccion centrada en x, y del array M1 de
    tamaño igual al array M2. Las coordenadas x,y son dadas en un sistema cartesiano centrado.

    Parameters
    ----------
    M1 : Array Numpy
        Arreglo del que se recorta una sección.
    M2 : Array Numpy
        Arreglo que determina el tamaño del recorte.
    x : Integer
        Coordenada en x del centro del recorte en un sistema cartesiano centrado para M1.
    y : Integer
        Coordenada en y del centro del recorte en un sistema cartesiano centrado para M1..

    Raises
    ------
    ValueError
        Si el tamaño del area a recortar o las coordenadas exceden el tamaño de la matriz original se disparara este error.

    Returns
    -------
    MC
        Arreglo de numpy con forma igual a M2.

    """
    (m,n)=M1.shape
    (mm,nn)=M2.shape
    if mm>m or nn>n:
        raise ValueError('La matriz recortada debe ser mas pequeña que la original')
    MC=M1[np.int32(np.ceil((m-mm)*0.5)+y):np.int32(np.ceil((m+mm)*0.5)+y),np.int32(np.ceil((n-nn)*0.5)+x):np.int32(np.ceil((n+nn)*0.5)+x)]
    return MC

def fusion(M1,M2,x,y):
    """
    Esta funcion permite insertar M2 a una seccion centrada en (x, y) del array M1.
    Las coordenadas x,y son dadas en un sistema cartesiano centrado.

    Parameters
    ----------
    M1 : Array Numpy
        Arreglo al que se inserta una sección.
    M2 : Array Numpy
        Arreglo a insertar.
    x : Integer
        Coordenada en x del centro del recorte en un sistema cartesiano centrado para M1.
    y : Integer
        Coordenada en y del centro del recorte en un sistema cartesiano centrado para M1..

    Raises
    ------
    ValueError
        Si el tamaño del area a insertar o las coordenadas exceden el tamaño de la matriz original se disparara este error.

    Returns
    -------
    MC
        Arreglo de numpy con forma igual a M1.

    """
    (m,n)=M1.shape
    (mm,nn)=M2.shape
    if mm>m or nn>n:
        raise ValueError('La matriz a instertar debe ser mas pequeña que la original')
    M1[np.int32(np.ceil((m-mm)*0.5)+y):np.int32(np.ceil((m+mm)*0.5)+y),np.int32(np.ceil((n-nn)*0.5)+x):np.int32(np.ceil((m+nn)*0.5)+x)]=M2
    MC=np.copy(M1)
    return MC

def ftfresnel(M,N,dx,wl,z):
    
    """
    Esta funcion genera una funcion de transferencia  de tamaño (n,m) para la transformada de fresnel con distancia z tamaño de píxel dx y longitud de onda wl.
    Usar tamaños cuadrados para evitar artefactos de anisotropia, o generar una funcion cuadrada de mayor tamaño y recortar el tamaño deseado. 
    El programa determina si se genera la funcion de transferencia directamente.
    
    Parameters
    ----------
    M : Integer
        Tamaño de la funcion de transferencia en pixeles en la direccion horizontal
    N : Integer
        Tamaño de la funcion de transferencia en pixeles en la direccion horizontal
    dx : Float
        Tamaño de píxel
    wl : Float
        Longitud de onda
    z : Float
        Distancia de propagacion.

    Returns
    -------
    H
        Arreglo de numpy con tamaño N,M

    """
    Lx=M*dx
    Ly=N*dx
    k=2*np.pi/wl
    x=np.arange(-Lx/2,Lx/2,dx)
    y=np.arange(-Ly/2,Ly/2,dx)
    fx=np.arange(-1/(2*dx),1/(2*dx),1/Lx)
    fy=np.arange(-1/(2*dx),1/(2*dx),1/Ly)
    zmax=(dx*Lx)/wl
    FX,FY=np.meshgrid(fx,fy)
    H=np.exp(-1j*wl*z*np.pi*(FX**2+FY**2))
    H=sft.fftshift(H)
    return H

def irfresnel(M,N,dx,wl,z):
    
    """
    Esta funcion genera una funcion de tranfserencia  de tamaño (n,m) para la transformada de fresnel con distancia z tamaño de píxel dx y longitud de onda wl.
    Usar tamaños cuadrados para evitar artefactos de anisotropia, o generar una funcion cuadrada de mayor tamaño y recortar el tamaño deseado. 
    El programa genera la funcion de transferencia por medio de la respuesta al impulso en base a los paramtros introducidos.
    
    Parameters
    ----------
    M : Integer
        Tamaño de la funcion de transferencia en pixeles en la direccion horizontal
    N : Integer
        Tamaño de la funcion de transferencia en pixeles en la direccion horizontal
    dx : Float
        Tamaño de píxel
    wl : Float
        Longitud de onda
    z : Float
        Distancia de propagacion.

    Returns
    -------
    H
        Arreglo de numpy con tamaño N,M

    """
    Lx=M*dx
    Ly=N*dx
    k=2*np.pi/wl
    x=np.arange(-Lx/2,Lx/2,dx)
    y=np.arange(-Ly/2,Ly/2,dx)
    fx=np.arange(-1/(2*dx),1/(2*dx),1/Lx)
    fy=np.arange(-1/(2*dx),1/(2*dx),1/Ly)
    zmax=(dx*Lx)/wl
    X,Y=np.meshgrid(x,y)
    h=(1/(1j*wl*z))*np.exp(1j*k/(2*z)*(X**2+Y**2))
    H=sft.fft2(sft.fftshift(h))*(dx**2)  
    return H

def corr2(A,B):
    """
    Coeficiente de correlación compara 2 matrices y nos dice elemento a elemento, que tanto se parecen
    
    Parametros :
     ----------
    A : Matrix
        Elemento de referencia
    B : Matrix
        Elemento a comparar
    
    Retorna
    e/s Float
    """
    e=0
    s1=0
    s2=0
    Am=np.mean(A)
    Bm=np.mean(B)
    [n,m]=np.shape(A)
    for i in range(0,n):
        for j in range(0,m):
            e=e+(A[i,j]-Am)*(B[i,j]-Bm)
            s1=s1+(A[i,j]-Am)**2
            s2=s2+(B[i,j]-Bm)**2
    s=np.sqrt(s1*s2)
    return e/s

def propFF(u1,L1,lam,z): # PROPAGADOR BASADO EN LA FUNCIÓN DE RESPUESTA AL IMPULSO
  # propagation - FRAUNHOFER PATTERN
  # assumes same x and y side lengths and
  # uniform sampling
  # u1 - source plane field
  # L - source and observation plane side length
  # lambda - wavelength
  # z - propagation distance
  # u2 - observation plane field
  # L2 - Oobservation plane side length
  N=len(u1[0,:]) 
  dx1=L1/N
  k=2*np.pi/lam
  L2=lam*z/dx1
  dx2=lam*z/L1
  x2=np.arange(-L2/2,L2/2,dx2)  
  X2, Y2 = np.meshgrid(x2, x2);
  c=1/(1j*lam*z)*np.exp(1j*k/(2*z)*(X2**2+Y2**2))
  u2=c*np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(u1)))*dx1**2
  return u2, L2

In [303]:
cd Hologramas Sergio

[WinError 2] El sistema no puede encontrar el archivo especificado: 'Hologramas Sergio'
C:\Users\Acer\Documents\Yo\Universidad\SEMESTRE\Semestre 8\óptica experimental\Informe #4\Hologramas Sergio


# HOLOGRAMA DE FOURIER

In [528]:
%matplotlib qt
#Con este codigo cargan su holograma
pil_im = im.open('Fourier 1.bmp')
I=pil_im
I=I.convert('L')
#Este convierte la imagen en un arreglo de numpy cuadrado, rellenando los bordes con 0. Esto facilita la computacion de propagaciones en frensel
I=np.asarray(I)
I=I*1.
I=I/np.max(np.max(I))
#E=I
[n,m]=I.shape
M=max(n,m)
print(n,m)
Q=np.zeros([M,M]) 
E=fusion(Q,I,0,0) 

#E seria nuestro querido holograma.

F=sft.ifftshift(sft.fft2(sft.fftshift(E))) ## El campo propasgado a una distancia f es exactamente una transformada de fourier, luego solo es devolverla

R=abs(F*np.conj(F))  #Amplitud del campo
R=R/np.max(np.max(R)) #Normalizar
R=fusion(R,np.zeros([50,50]),0,0) #??
Q=np.zeros([500,500])
R=tajada(R,Q,200,150)

#Grasficación
fig = plt.figure(dpi=600)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(E,cmap='gray')
ax = fig.add_subplot(1, 2, 2)
ax.imshow(R**0.2,cmap='gray')

dx=1.6e-6 #tamaño de pixel en metros

L=M*dx
wl=633e-9
zc=dx*L/wl
print(zc)




    


#Recuerden que obtienen el campo de la reconstruccion, que es complejo. Deben mostrar la intensidad del mismo.


1080 1920
0.007764928909952606


In [ ]:


for i in range(0,10):
    
    F=sft.ifftshift(sft.fft2(sft.fftshift(E)))
    z=0.0001+i*0.00001 #Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.
    Ft=ftfresnel(M,M,dx,wl,z)
    F=convft(F,Ft,dx)
    R=abs(F*np.conj(F))  #Amplitud del campo
    R=R/np.max(np.max(R)) #Normalizar
    R=fusion(R,np.zeros([100,100]),0,0) #??
    Q=np.zeros([500,500])
    R=tajada(I,Q,400,150)

    
    
    
    #Grasficación
    
    fig2 = plt.figure(dpi=600)
    ax = fig2.add_subplot(1, 2, 1)
    ax.imshow(E,cmap='gray')
    ax = fig2.add_subplot(1, 2, 2)
    ax.imshow(R**0.2,cmap='gray')
    

# Holograma de fresnel


In [531]:
#Con este codigo cargan su holograma
%matplotlib qt
I2 = im.open('Fresnel1.bmp')
I2=I2.convert('L')
#display(I) #Muestra el holograma
#Este convierte la imagen en un arreglo de numpy cuadrado, rellenando los bordes con 0. Esto facilita la computacion de propagaciones en frensel
I2=np.asarray(I2)
I2=I2*1.
I2=I2/np.max(np.max(I2))
#E=I
[n,m]=I2.shape
M=max(n,m)
print(n,m)
Q=np.zeros([m,m])
E=fusion(Q,I2,0,0)
#E seria vuestro holograma.

F=sft.ifftshift(sft.fft2(sft.fftshift(E))) #Cuando el plano de reconstruccion es muy lejano se puede "acercar" con una FT
ft=ftfresnel(M,M,dx,wl,0.000377) #Se genera el kernel de la propagacion
F=convft(F,ft,dx)  #Se propaga del plano de fourier al de reconstruccion
R=F*np.conj(F)  #Intensidad del campo
R=abs(R) #Se convierte el campo en un array de numeros reales
R=R/np.max(np.max(R)) #Se normaliza
R=fusion(R,np.zeros([50,50]),0,0)
Q=np.zeros([500,500])
R=tajada(R,Q,220,100)

fig = plt.figure(dpi=150)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(E,cmap='gray')
ax = fig.add_subplot(1, 2, 2)
ax.imshow(R**0.2,cmap='gray')
plt.show()


1080 1920


In [536]:

for i in range(0,20):
    
    F=sft.ifftshift(sft.fft2(sft.fftshift(E)))
    z=-0.0008*i #Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.
    Ft=ftfresnel(M,M,dx,wl,z)
    F=convft(F,Ft,dx)
    R=abs(F*np.conj(F))  #Amplitud del campo
    R=R/np.max(np.max(R)) #Normalizar
    R=fusion(R,np.zeros([100,100]),0,0) #??
    Q=np.zeros([250,250])
    R=tajada(R,Q,220,100)
    
    
    
    #Grasficación
    
    fig2 = plt.figure(dpi=600)
    ax = fig2.add_subplot(1, 1, 1)
    ax.imshow(R**0.2,cmap='gray')
    

In [456]:
%matplotlib qt
pil_im =im.open('bonus2.bmp')

I=pil_im
I=I.convert('L')
#display(I) #Muestra el holograma
#Este convierte la imagen en un arreglo de numpy cuadrado, rellenando los bordes con 0. Esto facilita la computacion de propagaciones en frensel
I=np.asarray(I)
I=I*1.
I=I/np.max(np.max(I))
#E=I
[n,m]=I.shape
M=max(n,m)
print(n,m)
Q=np.zeros([M,M]) 
E=fusion(Q,I,0,0) 

#E seria nuestro querido holograma.
F=sft.ifftshift(sft.fft2(sft.fftshift(E)))
z=0.00060+0*0.000005 #Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.
Ft=ftfresnel(M,M,dx,wl,z)
F=convft(F,Ft,dx)
R=abs(F*np.conj(F))  #Amplitud del campo
R=R/np.max(np.max(R)) #Normalizar
#R=fusion(R,np.zeros([100,100]),0,0) #??
Q=np.zeros([800,800])
R=tajada(R,Q,700,100)
R=R**0.2
ar=sk.measure.shannon_entropy(R)
#Grasficación
fig = plt.figure(dpi=600)

z=0.00060#Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.

ax = fig.add_subplot(1, 1, 1)
ax.imshow(R,cmap='gray')

dx=1.6e-6 #tamaño de pixel en metros

L=M*dx
wl=633e-9
zc=dx*L/wl
print(zc)



1080 1920
0.007764928909952606


# Creación de fitlros

In [530]:
from scipy import fftpack


pil_im =im.open('bonus1.bmp')

I=pil_im
I=I.convert('L')
#display(I) #Muestra el holograma
#Este convierte la imagen en un arreglo de numpy cuadrado, rellenando los bordes con 0. Esto facilita la computacion de propagaciones en frensel
I=np.asarray(I)
I=I*1.
I=I/np.max(np.max(I))
#E=I
[n,m]=I.shape
M=max(n,m)
print(n,m)
Q=np.zeros([M,M]) 
E=fusion(Q,I,0,0) 

#E seria nuestro querido holograma.
F=sft.ifftshift(sft.fft2(sft.fftshift(E)))
z=0.00060+0*0.000005 #Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.
Ft=ftfresnel(M,M,dx,wl,z)
F=convft(F,Ft,dx)
R=abs(F*np.conj(F))  #Amplitud del campo
R=R/np.max(np.max(R)) #Normalizar

Q=np.zeros([M,M])
print(M)
Q1=np.ones([500,500])
low_pass=fusion(Q,Q1,550,0)

low_pass_fft=sft.ifftshift(sft.fft2(sft.fftshift(low_pass)))
image1_np=sft.ifftshift(sft.fft2(sft.fftshift(R)))




#multiply both the images
F=np.multiply(image1_np,low_pass_fft)
T=np.multiply(image1_np,low_pass)
B = abs(sft.ifftshift(sft.ifft2(sft.ifftshift(F))))
A = abs(sft.ifftshift(sft.ifft2(sft.ifftshift(T))))

EF=sft.ifftshift(sft.fft2(sft.fftshift(R)))

Q=np.zeros([M,M])
print(M)
Q1=np.ones([500,500])
low_pass=fusion(Q,Q1,650,0)

C=np.multiply(R,low_pass)

fig = plt.figure(dpi=150)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(C**0.2,cmap='gray')
ax = fig.add_subplot(1, 2, 2)
ax.imshow(R**0.2,cmap='gray')
plt.show()



1080 1920
1920
1920


### Busqueda de plano de mejor reconstrución

In [440]:
I=pil_im
I=I.convert('L')
#display(I) #Muestra el holograma
#Este convierte la imagen en un arreglo de numpy cuadrado, rellenando los bordes con 0. Esto facilita la computacion de propagaciones en frensel
I=np.asarray(I)
I=I*1.
I=I/np.max(np.max(I))
#E=I
[n,m]=I.shape
M=max(n,m)
print(n,m)
Q=np.zeros([M,M]) 
E=fusion(Q,I,0,0) 


for i in range(0,10):
    
    F=sft.ifftshift(sft.fft2(sft.fftshift(E)))
    z=0.00060+i*0.000005 #Distancia de propagacion en metros. Prueben con propagaciones del orden de milimetros para reconstruir su holograma de Fresnel.
    Ft=ftfresnel(M,M,dx,wl,z)
    F=convft(F,Ft,dx)
    R=abs(F*np.conj(F))  #Amplitud del campo
    R=R/np.max(np.max(R)) #Normalizar
    R=fusion(R,np.zeros([100,100]),0,0) #??
    Q=np.zeros([800,800])
    R=tajada(R,Q,700,100)
    
    ar=sk.measure.shannon_entropy(F)
    print(ar)
    
    
    
    #Grasficación
    
    fig2 = plt.figure(dpi=600)
    ax = fig2.add_subplot(1, 1, 1)
    ax.imshow(R**0.2,cmap='gray')
    

1080 1920
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217
21.813781191217


C:\Users\Acer\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
